In [6]:
import re
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta, timezone
from matplotlib.ticker import FuncFormatter, MultipleLocator, NullLocator, MaxNLocator, AutoMinorLocator
import matplotlib.dates as mdates
import pandas as pd
import pytz

from pytz import timezone # New Line
eastern = timezone('US/Eastern')

option_descriptions = {
    '1': 'Today',
    '2': 'This week',
    '3': '30 days',
    '4': '90 days',
    '5': '180 days',
    '6': '1 year',
    '7': '5 years',
    '8': '10 years',
    '9': 'Max',
    '10': 'Custom'
}

def is_market_open():
    now = datetime.now(eastern)
    open_time = now.replace(hour=9, minute=30, second=0)
    close_time = now.replace(hour=16, minute=0, second=0)
    end_date = datetime.now(eastern)

     # Check if current time is within the market hours and it's a weekday
    if open_time < now < close_time and now.weekday() < 5:
        return True
    else:
        return False

def generate_chart():
    # User input for ticker
    ticker_symbol = input("Enter ticker symbol: ").upper()  # Convert ticker to uppercase
    ticker_obj = yf.Ticker(ticker_symbol)
    financials_quarterly = ticker_obj.quarterly_financials
    
    # Instantiate the ticker object with yfinance
    ticker = yf.Ticker(ticker_symbol)
    
    # Prompt user to select a predefined date range option
    print("Select a date range option:")
    print("1. Today")
    print("2. This Week")
    print("3. 1 Month")
    print("4. 3 Months")
    print("5. 6 Months")
    print("6. 1 Year")
    print("7. 5 Years")
    print("8. 10 Years")
    print("9. Max")
    print("10. Custom Date Range")

    option = input("Enter a number (1-9) for a predefined date range, or 10 for a custom range [default is 6]: ")
    if not option:
        option = '6'  # Default to 1 year if no input

    # Set default frequency
    frequency = "1d"

    # Calculate the date range based on the selected option
    end_date = datetime.now(eastern)
    
    # Assign a default value to total_days
    total_days = 0
        
    if option == '1': # Today
        start_date = end_date.replace(hour=9, minute=30, second=0) # Set to market opening time
        #start_date = end_date
        total_days = 1
        frequency = "60m"  # Set frequency to hourly for 'today'
    elif option == '2':
        start_date = end_date - timedelta(days=end_date.weekday())
        total_days = (end_date - start_date).days
        frequency = "60m"  # Set frequency to hourly for 'this week'
    elif option == '3':
        start_date = end_date - timedelta(days=30)
        total_days = (end_date - start_date).days
    elif option == '4':
        start_date = end_date - timedelta(days=90)
        total_days = (end_date - start_date).days
    elif option == '5':
        start_date = end_date - timedelta(days=180)
        total_days = (end_date - start_date).days
    elif option == '6':
        start_date = end_date - timedelta(days=365)
        total_days = (end_date - start_date).days
    elif option == '7':
        start_date = end_date - timedelta(days=5 * 365)
        total_days = (end_date - start_date).days
    elif option == '8':
        start_date = end_date - timedelta(days=10 * 365)
        total_days = (end_date - start_date).days
    elif option == '9':
        start_date = datetime(1980, 1, 1, tzinfo=pytz.utc)  # Set an arbitrary past date as the "Max" option start date
        total_days = (end_date - start_date).days
    if option != '10':
        if start_date.tzinfo is None:
            start_date = eastern.localize(start_date)
    elif option == '10':
      
        # Set default dates
        default_start_date = (datetime.now() - timedelta(days=365)).strftime("%m-%d-%Y")
        default_end_date = datetime.now().strftime("%m-%d-%Y")

        # Prompt user for custom date range
        start_date_input = input(f"Enter start date (MMDDYYYY or MM-DD-YYYY) [default is {default_start_date}]: ")
        end_date_input = input(f"Enter end date (MMDDYYYY or MM-DD-YYYY) [default is {default_end_date}]: ")

        # Use user input or default dates
        start_date = start_date_input.replace("-", "") if start_date_input else default_start_date  # Adjusted to remove dashes if present
        end_date = end_date_input.replace("-", "") if end_date_input else default_end_date  # Adjusted to remove dashes if present

        # Add dashes to the date input if not already present
        start_date = re.sub(r'(\d{2})(\d{2})(\d{4})', r'\1-\2-\3', start_date)
        end_date = re.sub(r'(\d{2})(\d{2})(\d{4})', r'\1-\2-\3', end_date)

        # Convert date strings to datetime objects and localize them to the US Eastern timezone (necessary for option 10, because user is entering strings and need to be converted to 'datetime' objects to assign timezones
        #In the other user options (from 1 to 9), the start and end dates are derived programmatically using the datetime and timedelta functions. These operations automatically yield datetime objects, so there's no need to convert from strings.
        start_date = eastern.localize(datetime.strptime(start_date, "%m-%d-%Y"))
        end_date = eastern.localize(datetime.strptime(end_date, "%m-%d-%Y"))
        
        # Calculate total days
        total_days = (end_date - start_date).days
    else:     
        print("Invalid option. Please select a valid option.")
        return
     
    # User input for x-axis increment option
    increment_option = input("Enter H for hourly, D for daily, W for weekly, M for monthly [Default is 'D']): ")
    if not increment_option:
        increment_option = 'D' # Default to daily if no input
    
    if increment_option.lower() == "h":
        frequency = "60m" 
    elif increment_option.lower() == "d":
        frequency = "1d"
    elif increment_option.lower() == "w":
        frequency = "1wk"
    elif increment_option.lower() == "m":
        frequency = "1mo"

    # Download stock data
    try:
        data = yf.download(ticker_symbol, start=start_date, end=end_date, interval=frequency)  
    except Exception as e:
        print(f"An error occurred: {e}")
    
    if data.empty:
        print(f"No data available for ticker: {ticker_symbol} for the specified date range.")
        return
      
    if option == '1': 
        data = data[data.index.date == datetime.now().date()]
    
    # Fetch the most recent ticker data irrespective of the market status
    ticker_data = yf.Ticker(ticker_symbol).history(period="1d").tail(1)
    
    # Check if the market is open using the function
    market_open = is_market_open()
    if market_open:
        print()
        print("Market is open!")
    else:
        print()
        print("Market is closed.")
    
    if 'Close' in ticker_data.columns:
        last_close_price = ticker_data['Close'].values[0]
        # Get the prior period adjusted close price
        prior_adj_close = data['Adj Close'].shift(1).iloc[-1]
    else:
        last_close_price = ticker_data['Open'].values[0]

    try:
        current_price = yf.Ticker(ticker_symbol).info['currentPrice']  # Fetch current price
    except Exception as e:
        print(f"An error occurred: {e}")

    year_high_price = yf.Ticker(ticker_symbol).info['fiftyTwoWeekHigh'] # Fetch 52 week high

    # Fetch company name
    company_name = yf.Ticker(ticker_symbol).info['longName']
    
    if market_open:
        try:
            change_in_price = current_price - prior_adj_close 
            print(f"Company Name: {company_name}")
            print("Current price:", current_price)
            print(f"Prior Close Price: ${prior_adj_close:.2f}")
            print(f"Change in price: ${change_in_price:.2f}")
            print(f"52 Week High: ${year_high_price:.2f}")
            display_price = current_price
        except Exception as e:  # General Exception
            print("Error fetching current price:", e)  # Print Exception details
            print(yf.Ticker(ticker_symbol).info)  # Print full ticker info
    else:
        display_price = last_close_price
        change_in_price = last_close_price - prior_adj_close
        print(f"Company Name: {company_name}")
        print(f"Last Close Price: ${last_close_price:.2f}")
        print(f"Prior Close Price: ${prior_adj_close:.2f}")
        print(f"Change in price: ${change_in_price:.2f}")
        print(f"52 Week High: ${year_high_price:.2f}")
    
    if option == '10':
        print(f"Option {option} (Custom Date Range: {start_date.strftime('%m-%d-%Y')} to {end_date.strftime('%m-%d-%Y')})")
    else:
        print(f"Option {option} ({option_descriptions[option]})")
        
    # Fetch stock info
    stock_info = yf.Ticker(ticker_symbol).info

    # Get fundamental information
    fundamental_data = {
        "Trailing PE": round(stock_info.get('trailingPE', 0.00), 2) if stock_info.get('trailingPE') else "N/A",
        "Forward PE": round(stock_info.get('forwardPE', 0.00), 2) if stock_info.get('forwardPE') else "N/A",
        "Price to Earnings Growth Ratio (PEG)": round(stock_info.get('pegRatio', 0.00), 2) if stock_info.get('pegRatio') else "N/A",
        "Annual Earnings Growth (YoY)": f"{stock_info.get('earningsGrowth', 0.00) * 100:.2f}%" if stock_info.get('earningsGrowth') is not None else "N/A",
        "Annual Revenue Growth (YoY)": f"{stock_info.get('revenueGrowth', 0.00) * 100:.2f}%" if stock_info.get('revenueGrowth') is not None else "N/A",
        "Quarterly Earnings Growth (YoY)": f"{stock_info.get('earningsQuarterlyGrowth', 0.00) * 100:.2f}%" if stock_info.get('earningsQuarterlyGrowth') is not None else "N/A",
        "Quarterly Revenue Growth (YoY)": f"{stock_info.get('quarterlyRevenueGrowthQuarterly', 0.00) * 100:.2f}%" if stock_info.get('quarterlyRevenueGrowthQuarterly') is not None else "N/A",  
    }

    # Print fundamental information
    print()
    print("\033[1m" + "Fundamental Info:" + "\033[0m") # Printing "Fundamental Info" in bold
    for key, value in fundamental_data.items():
        print(f"{key}: {value}")

    # Update the "Adj Close" value for the current session if the market is open
    if market_open and option != '1':  # We don't want to double-add the current data for the "Today" option
        try: 
            data.at[data.index[-1], 'Adj Close'] = current_price
            #print("Debug: 'Adj Close' values - Market Open:", data['Adj Close'])
        except Exception as e:  
            print("Error in Block:", e) 
    else:
        # Append the latest data point if the market is closed and the fetched date is not in the dataset
        if ticker_data.index[0].date() > data.index[-1].date():
            #data = data.append(ticker_data) -replacing this line with the below because pandas is depricating .append
            data = pd.concat([data, ticker_data])
        #print("Debug: 'Adj Close' values - Market Closed:", data['Adj Close'])

    # Calculate moving averages
    ma10 = data['Adj Close'].ewm(span=10).mean()
    ma21 = data['Adj Close'].ewm(span=21).mean()
    ma50 = data['Adj Close'].rolling(window=50).mean()
    ma200 = data['Adj Close'].ewm(span=200).mean()
    # Create a figure and subplot
    fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(16, 10), gridspec_kw={'height_ratios': [2.5, 1]}) # New Line
    
    # Configure the major and minor ticks on the x axis
    ax1.xaxis.set_major_locator(mdates.MonthLocator()) # Places a major tick every month
    ax1.xaxis.set_minor_locator(mdates.WeekdayLocator(interval=1)) # Places a minor tick every week
    # Taking the below line out for now, and swapping in line below it to try without %Y' 
    # ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d')) # Format the date for better readability
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d')) # Format the date for better readability

    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    # Configure the major and minor ticks on the y axis
    ax1.yaxis.set_major_locator(MaxNLocator(14)) # Places 12 major ticks evenly spaced along the y axis
    ax1.yaxis.set_minor_locator(AutoMinorLocator(2)) # Places a minor tick halfway between each pair of major ticks

    # Add faint gridlines to the subplots
    ax1.grid(which='both', linestyle=':', linewidth='0.5', color='silver')
    ax2.grid(which='both', linestyle=':', linewidth='0.5', color='silver')

    # Adjust the displayed price label based on whether the market is open
    price_label = f"{ticker_symbol} (Last: {display_price:.2f})"
  
    # Plot the Technical Info / Moving Averages on the first subplot
    line_price = ax1.plot(data['Adj Close'].index, data['Adj Close'], label=price_label)[0] 
    line_ma10 = ax1.plot(ma10.index, ma10, label=f"10-day EMA (Final: {ma10.iloc[-1]:.2f})")[0]
    line_ma21 = ax1.plot(ma21.index, ma21, label=f"21-day EMA (Final: {ma21.iloc[-1]:.2f})")[0]
    line_ma50 = ax1.plot(ma50.index, ma50, label=f"50-day SMA (Final: {ma50.iloc[-1]:.2f})")[0]
    line_ma200 = ax1.plot(ma200.index, ma200, label=f"200-day EMA (Final: {ma200.iloc[-1]:.2f})")[0]

    # List for Technical Legend Handles
    tech_legend_handles = [line_price, line_ma10, line_ma21, line_ma50, line_ma200]
    
    # Draw the technical legend first, fix the location to best
    tech_legend = ax1.legend(handles=tech_legend_handles, loc='best', title='Technical Info', fontsize='small')
    tech_legend.set_title('Technical Info', prop={'size': 'small'})
    # Add the technical legend as an artist. This will ensure it isn't overwritten by the next legend.
    ax1.add_artist(tech_legend)
    
    ax1.set_ylabel("Price (USD)")
    
    # Calculate color for each bar in volume plot based on price changes from the previous close
    volume_colors = np.where(data["Close"] > data["Close"].shift(1), 'g', 'r')

    # Create a copy of the original volume data before adjustments
    original_volume_data = data["Volume"].copy()
   
    # Decide whether to convert volume to millions or thousands
    if total_days <= 7:  # If 'Today' or 'This Week'
        volume_multiplier = 1_000  # Convert to thousands
        volume_unit = 'K'
    elif total_days < 180:  # Explicitly check for the range
        volume_multiplier = 1_000  # Convert to thousands
        volume_unit = 'K'
    else:
        volume_multiplier = 1_000_000  # Convert to millions
        volume_unit = 'M'
    
    # Convert volume using the original data
    data["Volume"] = original_volume_data / volume_multiplier
    
    # Plot the volume data on the second subplot with the determined colors
    ax2.bar(data.index, data['Volume'], color=volume_colors, alpha=0.3)
    ax2.set_ylabel(f'Volume ({volume_unit})')

    # Adjust x-axis date formatting based on date range
    if total_days <= 1:  # If 'Today'
        date_format = mdates.DateFormatter('%H:%M')  # Display hours and minutes
        ax1.xaxis.set_major_formatter(date_format)
    elif total_days <= 7:  # If 'This Week'
        ax1.xaxis.set_major_locator(mdates.DayLocator())
        ax1.xaxis.set_minor_locator(mdates.HourLocator(interval=6))  # One tick every 6 hours
        ax1.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d-%H'))  # Show month-day and hour
    elif total_days <= 31:  # If less than or equal to 31 days
        ax1.xaxis.set_major_locator(mdates.DayLocator(bymonthday=range(1, 32)))
        ax1.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))
        # Explicitly set rotation for both major and minor labels (label rotation not working)
        #for label in ax1.get_xticklabels(which='both'):
        #   label.set_rotation(45)
        #ax1.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.MONDAY))
        #ax1.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))
    elif total_days <= 365:  # If less than or equal to a year
        ax1.xaxis.set_major_locator(mdates.MonthLocator())
        ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    elif total_days <= 365 * 20:  # If more than a year
        ax1.xaxis.set_major_locator(mdates.YearLocator())
        ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    else:  # If more than a 20 years
        ax1.xaxis.set_major_locator(mdates.YearLocator(base=5))  # every 5 years
        ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

    # Reduce the number of minor x-ticks when the date range is large
    if total_days > 365:
        ax1.xaxis.set_minor_locator(NullLocator())

    # Automatically adjust the subplot parameters for better layout
    plt.tight_layout()

    # Show the plot
    plt.show()

generate_chart()

Enter ticker symbol:  amzn


Select a date range option:
1. Today
2. This Week
3. 1 Month
4. 3 Months
5. 6 Months
6. 1 Year
7. 5 Years
8. 10 Years
9. Max
10. Custom Date Range


Enter a number (1-9) for a predefined date range, or 10 for a custom range [default is 6]:  
Enter H for hourly, D for daily, W for weekly, M for monthly [Default is 'D']):  


[*********************100%%**********************]  1 of 1 completed

Market is open!
An error occurred: 404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v6/finance/quoteSummary/AMZN?modules=financialData&modules=quoteType&modules=defaultKeyStatistics&modules=assetProfile&modules=summaryDetail&ssl=true


HTTPError: 404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v6/finance/quoteSummary/AMZN?modules=financialData&modules=quoteType&modules=defaultKeyStatistics&modules=assetProfile&modules=summaryDetail&ssl=true

In [12]:
ticker = yf.Ticker("AAPL")
print(ticker.info.keys())

HTTPError: 404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v6/finance/quoteSummary/AAPL?modules=financialData&modules=quoteType&modules=defaultKeyStatistics&modules=assetProfile&modules=summaryDetail&ssl=true

In [3]:
# This line has no changes
!pip install matplotlib

In [4]:
!pip install --upgrade yfinance

     ---------------------------------------- 0.0/65.6 kB ? eta -:--:--
     ---------------------------------------- 65.6/65.6 kB 1.8 MB/s eta 0:00:00
  Using cached peewee-3.17.0-py3-none-any.whl
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.28
    Uninstalling yfinance-0.2.28:
      Successfully uninstalled yfinance-0.2.28


In [5]:
!pip install matplotlib pandas numpy pytz